## solution to 2.2.b) optional
First we load the image with `PIL` and convert it to an array with `numpy`

In [ ]:
from PIL import Image #Class from PIL module to handle images
import numpy as np # standard python module for numerical calculation
import matplotlib.pyplot as plt # standard python module for plotting
import treecorr # module to use TreeCorr's code

# define the correlation function - brute force

def corrKK(cat,rmin,rmax,nrbin):
        
    # function to calculate distance r
    def get_r(A=[],B=[]):
        return np.linalg.norm(np.array(A)-np.array(B))
    
    # function to get the bin for the estimator. It returns linear binning if inputed linear values,
    # logarithmic binning for inputed log values
    def binning(x,xmin,xBinsize):
        return int((x-xmin)/(xBinsize+10**(-5)))

    # initialize vars
    
    # define bin size in logarithm space
    rBinsize = np.log(rmax/rmin)/nrbin 
    
    # define array to store the correlation function 
    xikk = np.zeros(nrbin)
    
    # define array to count number of points in each bin; corresponds to the denominator in the above equation
    binCntr = xikk.copy()
    
    # calculate and store in xikk
    for ii,p1 in enumerate(cat):# catalog 1
        for jj,p2 in enumerate(cat):# catalog 2
            
            # avoid count twice over pairs
            if jj >= ii: continue
                
            # get the distance between pairs
            r=get_r(p1[:2],p2[:2])
            
            # check distance belongs to estimator range
            if r < rmin or r >= rmax:continue
            
            # if distance is within the relevant range store the correlation of points into bins
            xikk[binning(np.log(r),np.log(rmin),rBinsize)] += p1[-1]*p2[-1]
            binCntr[binning(np.log(r),np.log(rmin),rBinsize)] += 1  
            
    # normalize xikk
    for ii in range(nrbin):
        if binCntr[ii] == 0 :continue
        xikk[ii] = xikk[ii]/binCntr[ii]
            
    return [xikk,binCntr]

In [ ]:
img = Image.open("images/triforce.png") # import image

This image has as many pixels in x and y directions as its width and height 

In [ ]:
print("heigh:{}, width:{}".format(img.height,img.width))

So we can convert this image to an array having as many entries as width and height, but first let us change this image to gray scale with the method [convert](https://pillow.readthedocs.io/en/stable/)

In [ ]:
cboat=np.asarray(img.convert("L"))

the image now is an array for every row and col it has a color associated with the pixel. Let us watch this

In [ ]:
print(cboat[15:18])

We can visualize this array with `matplotlib.imshow`, to appreciate lets load a cmap in grey scale _gs_

In [ ]:
gs=plt.cm.get_cmap('binary').reversed() # color map to binary 
vs=plt.cm.get_cmap('viridis').reversed() # color map to viridis
fooimg=plt.imshow(cboat,cmap=gs,vmin=np.min(cboat),vmax=np.max(cboat))
plt.colorbar(fooimg);plt.show()

Even though we see white pixels causing no problem, it is since they have a value asociated which result in _noise_ so we better reduce those values. For example normalizing and substracting the mean value

In [ ]:
cb=cboat/np.max(cboat)
cb=cb-np.mean(cb)
cb=cb-cb[0]

now values of $c$ belong to [-1,1]

In [ ]:
cbmin = np.min(cb);
cbmax = np.max(cb);
print("min(kb)=%5.2f,max(kb)=%5.2f" %(cbmin,cbmax))

We take a look to the "enhanced" image array which may look very similar to eye

In [ ]:
fooimg=plt.imshow(cb,cmap=gs,vmin=cbmin,vmax=cbmax)
plt.colorbar(fooimg)
plt.show()

On one hand we are gonna work with gray scale level, but on the other hand notice we now are working with pixels distance instead of physical distance. So we want to create _RA_ and _DEC_ arrays for this case in pixel distance

In [ ]:
raPix=np.arange(img.width);
decPix=np.arange(img.height);

Furthermore, to handle every pixel from the image we need to give positions to them; maybe work with masks afterward. Then, we first create the _RA_ and _DEC_ with as many entries as our c-field has, i.e., the shape of _cb_

In [ ]:
cb.shape

therefore we need _RA_ and _DEC_ with 400 entries, since we have 400 pixels. We make a 1D array for c out of cb

In [ ]:
cb1d=np.reshape(cb,(-1))
print(cb1d.shape)

And for making those 400 entries with the same order, as a meshgrid flattened, we use `numpy.meshgrid` and then we flatten it

In [ ]:
RA,DEC=np.meshgrid(decPix,raPix)
RA=np.reshape(RA,(-1))
DEC=np.reshape(DEC,(-1))

We now have the ingredients for creat a catalog, as for TC or for brute force algorithm. Wheater one or another we define the minimal, maximal distances and number of bins for it

In [ ]:
# minimum distance for correlation in image
rminImg=1;

# maximum distance for correlation in image
rmaxImg=np.sqrt(2)*img.width;

# number of bins in distance for the correlation of c-field
nrbinImg=12

Since `corrKK` recieves a catalog with [ra,dec,k], we create one and run `corrKK` for the image

In [ ]:
%%time
catImgbf=np.transpose(np.reshape([RA,DEC,cb1d],(3,len(cb1d))))
xiImg,binCntrImg=corrKK(catImgbf,rminImg,rmaxImg,nrbinImg)

For TC we do not need to create a catalog as above but by means of `treecorr.Catalog` class, then create a `treecorr.KKCorrelation` instance and finally run the calculation of $\xi$ with `treecorr.KKCorrelation`. We recommend you to checkout [TC's documentation](https://rmjarvis.github.io/TreeCorr/_build/html/kk.html) for deeper insights

In [ ]:
catImg=treecorr.Catalog(ra=RA,dec=DEC,k=cb1d,ra_units='deg', dec_units='deg')
tcKKcorrImg=treecorr.KKCorrelation(sep_units="deg",min_sep=rminImg,max_sep=rmaxImg,nbins=nrbinImg)
tcKKcorrImg.process(catImg)

Once done with calculation, let us see the results

In [ ]:
plt.plot(tcKKcorrImg.rnom,tcKKcorrImg.xi,c="black",label="$\\xi_{TC}$")
plt.plot(tcKKcorrImg.rnom,xiImg,linestyle="dashed",marker=".",markersize=12,label="$\\xi_{BF}$",c="black")
plt.legend(frameon=False,fontsize=15);plt.xlabel("$r$")
#plt.xlim(0,100)
plt.show()

Notice the peaks are very similar, maybe only shifted wich can be a result araising from inner TC's methods for binning and smarter procedure for the calculation